In [1]:
# Loading python modules used in this notebook
#import pygments
#from pygments import highlight
#from pygments.lexers import MakefileLexer, CppLexer
#from pygments.formatters import HtmlFormatter
from IPython.core.display import display,HTML

# Setup of the GOTM/FABM model for Langtjern

## Compiling GOTM and FABM

Please follow the instructions at <https://github.com/gotm-model/code>. Please note that the provided instructions are only available for Linux systems at present. 

For Windows, a combination of Visual Studio, CMake and the Intel Fortran compiler should work with minimal modifications when CMake is setup to generate Visual Studio project files.

For the rest of this document, it is assumed that Linux is used in combination with CMake, using GCC-5.4. 

### Getting the source code

Credentials might be required for repositories. The text highlighted in a box should be run in a shell.

#### GOTM

Clone the source code to a directory of your choice.
<section>
   <div class="warn">
   git clone https://github.com/gotm-model/code.git
   <div/>
</section>

Switch to lake branch.
<section>
   <div class="warn">
   git checkout lake
   <div/>
</section>

#### FABM
Clone the source code to a directory of your choice.
<section>
   <div class="warn">
   git clone git@gitlab.au.dk:anbios/fabm-prognos.git
   <div/>
</section>

#### Langtjern setup
Clone the source code to a directory of your choice.
<section>
   <div class="warn">
   git clone git@gitlab.au.dh:anbios/PROGNOS.git
   <div/>
</section>

### Compiling

#### FABM

From within FABM source directory.

Out-of-source builds should be preferred. Neater.
Create a _./build_ directory in the root folder for FABM (or wherever you like but point to the right _/src_ folder in the _cmake_ command below:

<section>
   <div class="warn">
   mkdir build <br>
   cd build <br>
   cmake ../src -DFABM_HOST=gotm <br>
   make <br>
   make install
   <div/>
</section>

Note that make will create libfabm.a in the _.build_ directory. _make install_ will place the library and some other include files in the _~/local_ directory. The later is convenient to have all the builds for both FABM and GOTM in a single directory (that can be added to the PATH) thus simplifying linking.

#### GOTM

From within GOTM source directory.

Repeating the comments for the FABM compilation. Create a _./build_ directory in the root folder for GOTM. You will need to point _cmake_ to FABM root dir. Modify the cmake command below to point the the right locations if necessary:

<section>
   <div class="warn">
   mkdir build <br>
   cd build <br>
   cmake ../src -DFABM_BASE=../../fabm-gotm <br>
   make <br>
   make install
   <div/>
</section>

### Running GOTM from the "PROGNOS/Langtjern" folder.


GOTM and FABM can take a variety of inputs, depending on data availability. The format required is described in the [FABM repository](https://github.com/fabm-model/fabm/wiki/GOTM). In short, model setup relies on three different types of files:

1. **.xml** General setup file. The different settings in _.nml_ files (see next item) will be read from the _langtjern.xml_file inside the "_Langtjern_" folder.
1. **.nml** (nameless) files, to regulate the interaction between FABM and GOTM. And also to specify processes to be used within GOTM.
2. **.yaml** to specify which biogeochemical models will be used and their parameterizations.
3. **.dat** data files used to force and evaluate the model. They are in the format specified in the above link. Their are two basic flavors of data files: depth-dependent or independent.

In order to start running simulations with GOTM and FABM, some basic setup needs to be peformed first.

From the "_Langtjern_" folder in the PROGNOS repository, _editscenario.sh_ &mdash;a wrapper to a python script&mdash; needs to be run. This basically creates the <i>gotm_fabm.nml</i> and other nameless files in the "_Langtjern_" folder, filling with options read from the _Langtjern.xml_ file. 

<b>Important note:</b> The default option is NOT to use FABM when running GOTM. In order to change this *fabm_calc*  needs to be set to _.true._ in <i>gotm_fabm.nml</i>.

If you want to modify any setting there are two options:

1. Modify the _Langtjern.xml_ file and run _editscenario.sh_.
2. Modify the nameless files.

#### Setting up the meteo data

In order to run GOTM some basic meteo data are required. In this case, the file is called 
<i>meteo_file.dat</i> and should be located inside the _Langtjern_ folder. When instantiated through _editscenario.sh_ this file while be a copy of the file specified in  _langtjern.xml_. The file telling GOTM where the meteo data are located is _airsea.nml_.

In our particular setup, the meteo file was generated by running a [python jupyter notebook](http://nbviewer.jupyter.org/github/Lecheps/PROGNOS_data/blob/master/PROGNOS_preprocessing.ipynb) that loads data from [AquaMonitor](http://www.aquamonitor.no/Portal/) and puts in in the format required by GOTM/FABM.


In [2]:
def css_styling():
    styles = open("./styles/custom.css", "r").read()
    return HTML(styles)
css_styling()

In [3]:
%%javascript
$('<div id="toc"></div>').css({position: 'fixed', top: '120px', left: 0}).appendTo(document.body);
$.getScript('https://kmahelona.github.io/ipython_notebook_goodies/ipython_notebook_toc.js');

<IPython.core.display.Javascript object>